# Model input ERP format preparation 

In this notebook: 
- Necessary inputs
- Read all epochs
- Function to create dataframe with average mismatch response for all participants (needs to be transformed to function)
- Formatting dataframe as suitable model input

## Imports

In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from math import nan
from scipy import stats

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
import helper_functions as hf # library useful for eeg and erp data cleaning
import initialization_functions #library to import data
import epod_helper

In [2]:
metadata = pd.read_csv('metadata.csv', sep = ',')

In [3]:
metadata

,eeg_file,ParticipantID,test,sex,age_months,age_months_days,dyslexic_parent,Group_AccToParents,eeg_path,path_epoch,epoch_file
0,101a,101,a,m,20,20;22,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,101a_epo.fif
1,101b,101,b,m,23,23;16,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,101b_epo.fif
2,102a,102,a,f,20,20;27,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,102a_epo.fif
3,102b,102,b,f,23,23;16,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,102b_epo.fif
4,103a,103,a,f,20,20;23,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,103a_epo.fif
...,...,...,...,...,...,...,...,...,...,...,...
88,152a,152,a,f,19,19;9,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,152a_epo.fif
89,152b,152,b,f,22,22;8,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,152b_epo.fif
90,153a,153,a,m,20,20;22,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,153a_epo.fif
91,154a,154,a,m,20,20;9,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,F:\Stage\ePODIUM\Data\ePodium_projectfolder\ep...,154a_epo.fif


## Read all epochs from files

The function below loads all filtered epochs from the dataframe which contains the metadata and the epochs filepaths + filenames. The epochs are arrays for each stimuli with a time interval of -0.3 to 0.7.

In [4]:
def read_filtered_data(metadata):
    epochs = []
    for index, file in metadata.iterrows():
        print(f"Checking out file: {file['epoch_file']}")
        path = os.path.join(file['path_epoch'], file['epoch_file'])
        epoch = mne.read_epochs(path, preload=False)
        epochs.append(epoch)
    return epochs

In [5]:
epochs = read_filtered_data(metadata)

Checking out file: 101a_epo.fif
Reading F:\Stage\ePODIUM\Data\ePodium_projectfolder\epochs\101a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2266 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 101b_epo.fif
Reading F:\Stage\ePODIUM\Data\ePodium_projectfolder\epochs\101b_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2440 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 102a_epo.fif
Reading F:\Stage\ePODIUM\Data\ePodium_projectfolder\epochs\102a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2389 matching events found
No baseline correction applied
0 projection items activ

        0 CTF compensation matrices available
Not setting metadata
2440 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 115b_epo.fif
Reading F:\Stage\ePODIUM\Data\ePodium_projectfolder\epochs\115b_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2440 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 116a_epo.fif
Reading F:\Stage\ePODIUM\Data\ePodium_projectfolder\epochs\116a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2343 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 116b_epo.fif
Reading F:\Stage\ePODIUM\Data\ePodium_projectfolder\epochs\116b_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.2

Checking out file: 130a_epo.fif
Reading F:\Stage\ePODIUM\Data\ePodium_projectfolder\epochs\130a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2440 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 130b_epo.fif
Reading F:\Stage\ePODIUM\Data\ePodium_projectfolder\epochs\130b_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2440 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 131a_epo.fif
Reading F:\Stage\ePODIUM\Data\ePodium_projectfolder\epochs\131a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2440 matching events found
No baseline correction applied
0 projection items activ

        0 CTF compensation matrices available
Not setting metadata
2440 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 143a_epo.fif
Reading F:\Stage\ePODIUM\Data\ePodium_projectfolder\epochs\143a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2440 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 143b_epo.fif
Reading F:\Stage\ePODIUM\Data\ePodium_projectfolder\epochs\143b_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
1051 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 144a_epo.fif
Reading F:\Stage\ePODIUM\Data\ePodium_projectfolder\epochs\144a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.2

## Create pandas dataframe with the average difference between standard and deviant responses

The function below needs `metadata`, the loaded `epochs` and the definition of the standard and deviant events as input. You should define your standard and deviant events as an array. In the function `input_mmr_prep` it's important to know that the assumption is made that the deviant follows after a standard event. Therefore the deviant belonging to the standard is the  standard event number + 1. Make sure your events are numbered like this, else the function won't calculate the mismatch response.  

In [6]:
# define the events for standard and deviant
standard_events = [2,5,8,11]
deviant_events = [3,6,9,12]

def input_mmr_prep(metadata, epochs, standard_events): 
    # create dataframe with expected columns 
    df = pd.DataFrame(columns=["eeg_file", "paradigm", "channel", "mean"])

    # loop over all participants
    for i in range(len(metadata['eeg_file'])):
        
        # loop over every paradigm per participant 
        for j in standard_events: 
            paradigm = j
            # select the standard and deviant for a specific sequence and calculate the evoked response
            std_evoked = epochs[i][j].average() 
            dev_evoked = epochs[i][j+1].average()
            
            # calculate the mismatch response between standard and deviant evoked
            evoked_diff = mne.combine_evoked([std_evoked, dev_evoked], weights=[1, -1])
            
            # get a list of all channels
            chnames_list = evoked_diff.info['ch_names']
            
            # compute for every channel the distance mean of the mismatch line
            for channel in chnames_list: 
                chnames = mne.pick_channels(evoked_diff.info['ch_names'], include=[channel])
                roi_dict = dict(left_ROI=chnames) # unfortunately combine_channels only takes a dictionary as input
                roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
                mmr = roi_evoked.to_data_frame()
                mmr_avg = mmr['left_ROI'].mean()
                mmr_std = mmr['left_ROI'].std()
                mmr_skew = mmr['left_ROI'].skew()
                mmr_var = mmr['left_ROI'].var()
                mmr_kurt = mmr['left_ROI'].kurtosis()
                
                df = df.append({'eeg_file': metadata['eeg_file'][i], 'paradigm' : paradigm, 'channel': channel, 'mean' :  mmr_avg, 'std' : mmr_std, 'skew' : mmr_skew, 'kurt' : mmr_kurt, 'var' : mmr_var}, ignore_index=True) 
    return df

In [7]:
df = input_mmr_prep(metadata, epochs, standard_events)

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoke

...

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000229F9B37C10>, axis=0)".
  roi_evoke

In [8]:
df

,eeg_file,paradigm,channel,mean,kurt,skew,std,var
0,101a,2,Fp1,-11.094700,0.296626,-0.971012,26.227893,687.902370
1,101a,2,AF3,-39.574672,-0.983185,-0.198931,35.839026,1284.435783
2,101a,2,F7,-11.122825,0.673605,-0.693384,27.219345,740.892742
3,101a,2,F3,-44.410338,-0.862122,0.122667,36.690573,1346.198141
4,101a,2,FC1,-43.448037,-0.423392,0.623608,37.169785,1381.592941
...,...,...,...,...,...,...,...,...
11899,154b,11,F8,14.664999,0.135892,-0.317341,21.344284,455.578472
11900,154b,11,AF4,7.029734,-0.801619,-0.262314,26.564387,705.666676
11901,154b,11,Fp2,17.101891,-0.802646,0.117417,25.529122,651.736048
11902,154b,11,Fz,-7.372934,-0.122074,-0.423678,32.889471,1081.717310


In [9]:
df.columns

Index(['eeg_file', 'paradigm', 'channel', 'mean', 'kurt', 'skew', 'std',
       'var'],
      dtype='object')

## Transpose dataframe into combination of paradigm and channel per participant

We now want a single row for every participant containing the paradigm and corresponding channels. The code below generates this dataframe. 

In [10]:
# transformation of the dataframe
df = df.pivot(index='eeg_file', columns=['paradigm','channel'])
df

mean                                                          \
paradigm         2                                                            
channel         Fp1        AF3         F7         F3        FC1         FC5   
eeg_file                                                                      
101a     -11.094700 -39.574672 -11.122825 -44.410338 -43.448037  -19.790442   
101b     -55.456027 -51.474125 -46.160871 -81.619369 -48.458636 -103.995696   
102a     -10.255731   4.886056  -4.822772   2.250893  14.713139   -1.191946   
102b      12.343595 -14.385865 -25.882261   0.045743  21.888221   -5.243899   
103a     -42.862750 -20.541033 -46.932457 -12.579665  -8.396244  -33.579034   
...             ...        ...        ...        ...        ...         ...   
152a     -22.214103 -23.250318  10.652680 -12.656583 -29.704682   15.358242   
152b     -68.975461 -59.533007 -73.920828 -58.495751 -32.487219  -46.924031   
153a      20.415092  11.310084  27.815455  15.665279  28.298770   36.961245   
154a     -25.032477 -41.317576 -29.600080 -21.298110 -11.713829   -5.765159   
154b     -22.187153 -18.729368 -22.906843 -15.561530  -5.666525   -0.550179   

                                                      ...           var  \
paradigm                                              ...            11   
channel          T7         C3        CP1        CP5  ...            C4   
eeg_file                                              ...                 
101a     -25.778037 -30.380207 -11.238418 -41.946488  ...    762.670226   
101b     -24.407061 -50.099087 -40.616175 -11.810666  ...   2165.326248   
102a      -7.311456   6.309722  18.437361  -5.227678  ...    761.048582   
102b      -8.564132   5.209367   3.551395  17.702354  ...    590.989731   
103a     -24.454217 -17.723062 -21.572698 -27.931529  ...   1852.563978   
...             ...        ...        ...        ...  ...           ...   
152a       6.039440 -26.762907 -30.926283 -11.823460  ...   1617.153496   
152b     -27.338126 -14.222934 -15.477383 -25.981824  ...  16502.378319   
153a      42.726564  55.959136  41.946700  48.287420  ...   1245.396641   
154a      -7.185179   2.254453   2.927867   1.459721  ...   1014.501828   
154b       1.853320  10.057513  18.224220   7.060183  ...    362.934460   

                                                                  \
paradigm                                                           
channel             T8           FC6           FC2            F4   
eeg_file                                                           
101a        522.288319    503.579841    730.976986    784.869826   
101b       2376.740370   2423.607068   1983.181446   6267.401202   
102a        727.940573    941.338939   1208.099385   1280.764121   
102b        972.166255    628.327540    962.675997    984.813514   
103a        872.173084   1249.377675    900.424680    832.126843   
...                ...           ...           ...           ...   
152a       1268.170904   1044.872434   1151.757620    947.203355   
152b      17670.234963  16765.247100  15969.908577  16357.050465   
153a       2497.629661   5778.741833   2283.230102   3776.204198   
154a        944.301391   1465.571591   1371.937348   2557.716138   
154b        592.014598    461.144044    956.374284    947.722850   

                                                                                
paradigm                                                                        
channel             F8           AF4           Fp2            Fz            Cz  
eeg_file                                                                        
101a        721.226241   1370.065441    760.620246    929.182064   1065.772537  
101b       1767.575544   2700.982359   2681.020293   2189.221655   2835.964994  
102a       1490.128614   2036.161613   1843.434216   1658.302136   1515.948233  
102b        863.933296   1240.417567    312.562222   1032.612092    369.316454  
103a        645.467029    564.4363

In [11]:
df.columns = ['_'.join(str(s).strip() for s in col if s) for col in df.columns]
df

,mean_2_Fp1,mean_2_AF3,mean_2_F7,mean_2_F3,mean_2_FC1,mean_2_FC5,mean_2_T7,mean_2_C3,mean_2_CP1,mean_2_CP5,...,var_11_C4,var_11_T8,var_11_FC6,var_11_FC2,var_11_F4,var_11_F8,var_11_AF4,var_11_Fp2,var_11_Fz,var_11_Cz
eeg_file,,,,,,,,,,,,,,,,,,,,,
101a,-11.094700,-39.574672,-11.122825,-44.410338,-43.448037,-19.790442,-25.778037,-30.380207,-11.238418,-41.946488,...,762.670226,522.288319,503.579841,730.976986,784.869826,721.226241,1370.065441,760.620246,929.182064,1065.772537
101b,-55.456027,-51.474125,-46.160871,-81.619369,-48.458636,-103.995696,-24.407061,-50.099087,-40.616175,-11.810666,...,2165.326248,2376.740370,2423.607068,1983.181446,6267.401202,1767.575544,2700.982359,2681.020293,2189.221655,2835.964994
102a,-10.255731,4.886056,-4.822772,2.250893,14.713139,-1.191946,-7.311456,6.309722,18.437361,-5.227678,...,761.048582,727.940573,941.338939,1208.099385,1280.764121,1490.128614,2036.161613,1843.434216,1658.302136,1515.948233
102b,12.343595,-14.385865,-25.882261,0.045743,21.888221,-5.243899,-8.564132,5.209367,3.551395,17.702354,...,590.989731,972.166255,628.327540,962.675997,984.813514,863.933296,1240.417567,312.562222,1032.612092,369.316454
103a,-42.862750,-20.541033,-46.932457,-12.579665,-8.396244,-33.579034,-24.454217,-17.723062,-21.572698,-27.931529,...,1852.563978,872.173084,1249.377675,900.424680,832.126843,645.467029,564.436353,605.366144,745.656255,1906.713129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152a,-22.214103,-23.250318,10.652680,-12.656583,-29.704682,15.358242,6.039440,-26.762907,-30.926283,-11.823460,...,1617.153496,1268.170904,1044.872434,1151.757620,947.203355,854.358568,13320.282929,1019.639385,1447.158460,1093.914837
152b,-68.975461,-59.533007,-73.920828,-58.495751,-32.487219,-46.924031,-27.338126,-14.222934,-15.477383,-25.981824,...,16502.378319,17670.234963,16765.247100,15969.908577,16357.050465,17317.145666,15873.698577,15380.916535,17980.725599,14749.145228
153a,20.415092,11.310084,27.815455,15.665279,28.298770,36.961245,42.726564,55.959136,41.946700,48.287420,...,1245.396641,2497.629661,5778.741833,2283.230102,3776.204198,2495.329787,4000.614745,5975.722219,3299.226404,1579.169646


In [12]:
df.reset_index(inplace=True)
df

,eeg_file,mean_2_Fp1,mean_2_AF3,mean_2_F7,mean_2_F3,mean_2_FC1,mean_2_FC5,mean_2_T7,mean_2_C3,mean_2_CP1,...,var_11_C4,var_11_T8,var_11_FC6,var_11_FC2,var_11_F4,var_11_F8,var_11_AF4,var_11_Fp2,var_11_Fz,var_11_Cz
0,101a,-11.094700,-39.574672,-11.122825,-44.410338,-43.448037,-19.790442,-25.778037,-30.380207,-11.238418,...,762.670226,522.288319,503.579841,730.976986,784.869826,721.226241,1370.065441,760.620246,929.182064,1065.772537
1,101b,-55.456027,-51.474125,-46.160871,-81.619369,-48.458636,-103.995696,-24.407061,-50.099087,-40.616175,...,2165.326248,2376.740370,2423.607068,1983.181446,6267.401202,1767.575544,2700.982359,2681.020293,2189.221655,2835.964994
2,102a,-10.255731,4.886056,-4.822772,2.250893,14.713139,-1.191946,-7.311456,6.309722,18.437361,...,761.048582,727.940573,941.338939,1208.099385,1280.764121,1490.128614,2036.161613,1843.434216,1658.302136,1515.948233
3,102b,12.343595,-14.385865,-25.882261,0.045743,21.888221,-5.243899,-8.564132,5.209367,3.551395,...,590.989731,972.166255,628.327540,962.675997,984.813514,863.933296,1240.417567,312.562222,1032.612092,369.316454
4,103a,-42.862750,-20.541033,-46.932457,-12.579665,-8.396244,-33.579034,-24.454217,-17.723062,-21.572698,...,1852.563978,872.173084,1249.377675,900.424680,832.126843,645.467029,564.436353,605.366144,745.656255,1906.713129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,152a,-22.214103,-23.250318,10.652680,-12.656583,-29.704682,15.358242,6.039440,-26.762907,-30.926283,...,1617.153496,1268.170904,1044.872434,1151.757620,947.203355,854.358568,13320.282929,1019.639385,1447.158460,1093.914837
89,152b,-68.975461,-59.533007,-73.920828,-58.495751,-32.487219,-46.924031,-27.338126,-14.222934,-15.477383,...,16502.378319,17670.234963,16765.247100,15969.908577,16357.050465,17317.145666,15873.698577,15380.916535,17980.725599,14749.145228
90,153a,20.415092,11.310084,27.815455,15.665279,28.298770,36.961245,42.726564,55.959136,41.946700,...,1245.396641,2497.629661,5778.741833,2283.230102,3776.204198,2495.329787,4000.614745,5975.722219,3299.226404,1579.169646
91,154a,-25.032477,-41.317576,-29.600080,-21.298110,-11.713829,-5.765159,-7.185179,2.254453,2.927867,...,1014.501828,944.301391,1465.571591,1371.937348,2557.716138,2172.866537,1395.610995,951.510805,2170.049024,1396.995268


## Merge and safe dataframe

We still need to merge some of the metadata into the dataframe, so we have the information of the age, gender and label of the participant. 

In [13]:
df = pd.merge(df, metadata, on='eeg_file')

In [14]:
pd.set_option('display.max_columns', None)

In [15]:
df

,eeg_file,mean_2_Fp1,mean_2_AF3,mean_2_F7,mean_2_F3,mean_2_FC1,mean_2_FC5,mean_2_T7,mean_2_C3,mean_2_CP1,mean_2_CP5,mean_2_P7,mean_2_P3,mean_2_Pz,mean_2_PO3,mean_2_O1,mean_2_Oz,mean_2_O2,mean_2_PO4,mean_2_P4,mean_2_P8,mean_2_CP6,mean_2_CP2,mean_2_C4,mean_2_T8,mean_2_FC6,mean_2_FC2,mean_2_F4,mean_2_F8,mean_2_AF4,mean_2_Fp2,mean_2_Fz,mean_2_Cz,mean_5_Fp1,mean_5_AF3,mean_5_F7,mean_5_F3,mean_5_FC1,mean_5_FC5,mean_5_T7,mean_5_C3,mean_5_CP1,mean_5_CP5,mean_5_P7,mean_5_P3,mean_5_Pz,mean_5_PO3,mean_5_O1,mean_5_Oz,mean_5_O2,mean_5_PO4,mean_5_P4,mean_5_P8,mean_5_CP6,mean_5_CP2,mean_5_C4,mean_5_T8,mean_5_FC6,mean_5_FC2,mean_5_F4,mean_5_F8,mean_5_AF4,mean_5_Fp2,mean_5_Fz,mean_5_Cz,mean_8_Fp1,mean_8_AF3,mean_8_F7,mean_8_F3,mean_8_FC1,mean_8_FC5,mean_8_T7,mean_8_C3,mean_8_CP1,mean_8_CP5,mean_8_P7,mean_8_P3,mean_8_Pz,mean_8_PO3,mean_8_O1,mean_8_Oz,mean_8_O2,mean_8_PO4,mean_8_P4,mean_8_P8,mean_8_CP6,mean_8_CP2,mean_8_C4,mean_8_T8,mean_8_FC6,mean_8_FC2,mean_8_F4,mean_8_F8,mean_8_AF4,mean_8_Fp2,mean_8_Fz,mean_8_Cz,mean_11_Fp1,mean_11_AF3,mean_11_F7,mean_11_F3,mean_11_FC1,mean_11_FC5,mean_11_T7,mean_11_C3,mean_11_CP1,mean_11_CP5,mean_11_P7,mean_11_P3,mean_11_Pz,mean_11_PO3,mean_11_O1,mean_11_Oz,mean_11_O2,mean_11_PO4,mean_11_P4,mean_11_P8,mean_11_CP6,mean_11_CP2,mean_11_C4,mean_11_T8,mean_11_FC6,mean_11_FC2,mean_11_F4,mean_11_F8,mean_11_AF4,mean_11_Fp2,mean_11_Fz,mean_11_Cz,kurt_2_Fp1,kurt_2_AF3,kurt_2_F7,kurt_2_F3,kurt_2_FC1,kurt_2_FC5,kurt_2_T7,kurt_2_C3,kurt_2_CP1,kurt_2_CP5,kurt_2_P7,kurt_2_P3,kurt_2_Pz,kurt_2_PO3,kurt_2_O1,kurt_2_Oz,kurt_2_O2,kurt_2_PO4,kurt_2_P4,kurt_2_P8,kurt_2_CP6,kurt_2_CP2,kurt_2_C4,kurt_2_T8,kurt_2_FC6,kurt_2_FC2,kurt_2_F4,kurt_2_F8,kurt_2_AF4,kurt_2_Fp2,kurt_2_Fz,kurt_2_Cz,kurt_5_Fp1,kurt_5_AF3,kurt_5_F7,kurt_5_F3,kurt_5_FC1,kurt_5_FC5,kurt_5_T7,kurt_5_C3,kurt_5_CP1,kurt_5_CP5,kurt_5_P7,kurt_5_P3,kurt_5_Pz,kurt_5_PO3,kurt_5_O1,kurt_5_Oz,kurt_5_O2,kurt_5_PO4,kurt_5_P4,kurt_5_P8,kurt_5_CP6,kurt_5_CP2,kurt_5_C4,kurt_5_T8,kurt_5_FC6,kurt_5_FC2,kurt_5_F4,kurt_5_F8,kurt_5_AF4,kurt_5_Fp2,kurt_5_Fz,kurt_5_Cz,kurt_8_Fp1,kurt_8_AF3,kurt_8_F7,kurt_8_F3,kurt_8_FC1,kurt_8_FC5,kurt_8_T7,kurt_8_C3,kurt_8_CP1,kurt_8_CP5,kurt_8_P7,kurt_8_P3,kurt_8_Pz,kurt_8_PO3,kurt_8_O1,kurt_8_Oz,kurt_8_O2,kurt_8_PO4,kurt_8_P4,kurt_8_P8,kurt_8_CP6,kurt_8_CP2,kurt_8_C4,kurt_8_T8,kurt_8_FC6,kurt_8_FC2,kurt_8_F4,kurt_8_F8,kurt_8_AF4,kurt_8_Fp2,kurt_8_Fz,kurt_8_Cz,kurt_11_Fp1,kurt_11_AF3,kurt_11_F7,kurt_11_F3,kurt_11_FC1,kurt_11_FC5,kurt_11_T7,kurt_11_C3,kurt_11_CP1,kurt_11_CP5,kurt_11_P7,kurt_11_P3,kurt_11_Pz,kurt_11_PO3,kurt_11_O1,kurt_11_Oz,kurt_11_O2,kurt_11_PO4,kurt_11_P4,kurt_11_P8,kurt_11_CP6,kurt_11_CP2,kurt_11_C4,kurt_11_T8,kurt_11_FC6,kurt_11_FC2,kurt_11_F4,kurt_11_F8,kurt_11_AF4,kurt_11_Fp2,kurt_11_Fz,kurt_11_Cz,skew_2_Fp1,skew_2_AF3,skew_2_F7,skew_2_F3,skew_2_FC1,skew_2_FC5,skew_2_T7,skew_2_C3,skew_2_CP1,skew_2_CP5,skew_2_P7,skew_2_P3,skew_2_Pz,skew_2_PO3,skew_2_O1,skew_2_Oz,skew_2_O2,skew_2_PO4,skew_2_P4,skew_2_P8,skew_2_CP6,skew_2_CP2,skew_2_C4,skew_2_T8,skew_2_FC6,skew_2_FC2,skew_2_F4,skew_2_F8,skew_2_AF4,skew_2_Fp2,skew_2_Fz,skew_2_Cz,skew_5_Fp1,skew_5_AF3,skew_5_F7,skew_5_F3,skew_5_FC1,skew_5_FC5,skew_5_T7,skew_5_C3,skew_5_CP1,skew_5_CP5,skew_5_P7,skew_5_P3,skew_5_Pz,skew_5_PO3,skew_5_O1,skew_5_Oz,skew_5_O2,skew_5_PO4,skew_5_P4,skew_5_P8,skew_5_CP6,skew_5_CP2,skew_5_C4,skew_5_T8,skew_5_FC6,skew_5_FC2,skew_5_F4,skew_5_F8,skew_5_AF4,skew_5_Fp2,skew_5_Fz,skew_5_Cz,skew_8_Fp1,skew_8_AF3,skew_8_F7,skew_8_F3,skew_8_FC1,skew_8_FC5,skew_8_T7,skew_8_C3,skew_8_CP1,skew_8_CP5,skew_8_P7,skew_8_P3,skew_8_Pz,skew_8_PO3,skew_8_O1,skew_8_Oz,skew_8_O2,skew_8_PO4,skew_8_P4,skew_8_P8,skew_8_CP6,skew_8_CP2,skew_8_C4,skew_8_T8,skew_8_FC6,skew_8_FC2,skew_8_F4,skew_8_F8,skew_8_AF4,skew_8_Fp2,skew_8_Fz,skew_8_Cz,skew_11_Fp1,skew_11_AF3,skew_11_F7,skew_11_F3,skew_11_FC1,skew_11_FC5,skew_11_T7,skew_11_C3,skew_11_CP1,skew_11_CP5,skew_11_P7,skew_11_P3,skew_11_Pz,skew_11_PO3,skew_11_O1,skew_11_Oz,skew_11_O2,skew_11_PO4,skew_11_P4,skew_11_P8,skew_11_CP6,skew_11_CP2,

Drop some unnecessary columns. 

In [16]:
df = df.drop(['eeg_file','age_months_days',
       'dyslexic_parent', 'eeg_path', 'path_epoch',
       'epoch_file'], axis =1)

Every participant has now two rows, one for each test. We now want a single row for each participant, since the participant already has experience with the test and therefore the test at time b might differ from the test atime a. To avoid bias in the data, both tests will be appended to the participant into one row. 

In [17]:
a = df.loc[df['test'] == 'a']
b = df.loc[df['test'] == 'b']
a = a.drop(['test'],axis=1)
b = b.drop(['test','sex','age_months','Group_AccToParents'], axis=1)

In [18]:
df = pd.merge(a, b, on='ParticipantID',suffixes=("_a", "_b"))
df = df.drop(['ParticipantID'],axis=1)

In [19]:
df['sex'] = np.where((df['sex']=='m'), 1,0)
df['Group_AccToParents'] = np.where((df['Group_AccToParents']=='At risk'), 1,0)

In [20]:
first = df.pop('Group_AccToParents')
df.insert(0, 'Group_AccToParents', first)

To remove some outliers on the data, the z score is calculated. 

In [21]:
#df[(np.abs(stats.zscore(df)) < 5).all(axis=1)] # z score 

In [22]:
df.to_csv('df_avg_mmr.csv', index=False) # safe dataframe

## PCA analysis on feature reduction 

In [23]:
X = df.drop('Group_AccToParents',1)
y = df['Group_AccToParents']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [25]:
pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [26]:
explained_variance = pca.explained_variance_ratio_

In [27]:
explained_variance

array([5.86974982e-01, 2.81254941e-01, 1.31754342e-01, 1.51642371e-05,
       3.28382568e-07, 8.30018405e-08, 5.62716219e-08, 3.04694123e-08,
       2.34329562e-08, 1.51867800e-08, 1.14221844e-08, 5.87935459e-09,
       4.69729525e-09, 3.30146901e-09, 2.58201886e-09, 2.19576086e-09,
       7.81090503e-10, 5.57723376e-10, 4.73530539e-10, 3.53142470e-10,
       2.96164600e-10, 2.57199630e-10, 1.57298387e-10, 1.27051997e-10,
       1.13265130e-10, 6.39972827e-11, 4.58489006e-11, 3.58717631e-11,
       3.45622585e-11, 2.55086206e-11, 2.03185451e-11, 1.54108517e-11,
       4.22258271e-33])

In [28]:
classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [29]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

[[3 3]
 [2 1]]
0.4444444444444444


In [30]:
X.shape

(42, 1282)